# Calculating XY Tool Offsets Helper Notebook
This notebook scaffolds the steps outlined [here](https://www.jubilee3d.com/index.php?title=Setting_Tool_Offsets) to align the XY offset of tools. Some of these steps are manual, and others require sending commands. This notebook will walk through all necessary steps!

This notebook uses a USB microscope to manually align tools; see the link above for more information.

In [7]:
from science_jubilee.Machine import Machine
from science_jubilee.tools.Tool import Tool
import os
import cv2 as cv2

In [8]:
# This function will help us center our tools with a camera stream
def camera_stream(cap):
    center = None
    while center is None:
        # the first frame grab is sometimes empty
        ret, frame = cap.read()
        h, w = frame.shape[0:2]
        center = (int(w/2), int(h/2))

    while True:
        ret, frame = cap.read()
        target = cv2.circle(frame, center, 5, (0,255,0), -1)
        cv2.imshow('Input', frame)
        c = cv2.waitKey(1)
        if c ==27: #27 is the built in code for ESC so press escape to close the window. 
            break 

    cap.release()
    cv2.destroyAllWindows()

In [6]:
# Establish connection with machine
m = Machine(address="jubilee.local") # edit the address as necessary

In [ ]:
# Make sure no XY tool offsets are set for your second tool
# TODO: Add XY offset info to Machine.py
# For now, send G10 P<your_tool_number> X0 Y0 from DWC to clear all offsets

In [ ]:
# Pick up the tool and move it to a reference position (e.g. 150, 150)
tool_number = 4 # Which tool are you setting the offsets for?
tool_name = 'test' # A human readable name for this tool
tool = Tool(tool_number, tool_name)
m.load_tool(tool)
m.pickup_tool(tool)
m.moveTo(x=150, y=150) # approx location on magnetic bed


In [ ]:
# Move the bed down and install the USB microscope onto the bed facing up
m.move_to(z=150)

In [ ]:
# Open a live video window and center the z-probe using DWC
# Press Esc to stop streaming
cap = cv2.VideoCapture(0) # Note that the index corresponding to your camera may not be zero
                          # but this is the most common default. Try 1, 2, etc if 0 doesn't work

camera_stream(cap)

In [ ]:
# Record the coordinates at which the z-probe is centered
pos = m.get_position()
zprobe_x = float(pos['X'])
zprobe_y = float(pos['Y'])
print(zprobe_x, zprobe_y) # using a height of 150 with tool equipped

In [ ]:
# Open a live feed again, and center the tip of the equipped tool using DWC
cap = cv2.VideoCapture(0) # Edit this index again as needed
camera_stream(cap)

In [ ]:
# record these coordinates
pos = m.get_position()
tool_x = float(pos['X'])
tool_y = float(pos['Y'])
print(tool_x, tool_y)

In [ ]:
# Sbtract your tool (x,y) coords from the probe (x,y) coords - this is your XY tool offset
# Add the following command to your Toffsets.g file
xoff = zprobe_x - tool_x
yoff = zprobe_y - tool_y
tool_number = 1
print(f"G10 P{tool_number} X{xoff:.2f} Y{yoff:.2f}")